In [3]:
import numpy as np
import napari

import matplotlib.pyplot as plt

from skimage.io import imread, imsave
from skimage import exposure

from aicsimageio.readers.tiff_reader import TiffReader

from cellpose import models, io

from tqdm.notebook import tqdm

In [4]:
from cellpose.io import logger_setup
logger, logfile = logger_setup()

creating new log file
2022-05-23 14:02:22,704 [INFO] WRITING LOG OUTPUT TO C:\Users\KarenDuffGroup\.cellpose\run.log


## Start napari

In [5]:
viewer = napari.Viewer()

C:\Users\KarenDuffGroup\AppData\Roaming\Python\Python38\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


2022-05-23 14:02:27,100 [WARNING] QWindowsWindow::setGeometry: Unable to set geometry 1400x1370+0+20 (frame: 1422x1426-11-25) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 2102x2058+3+34 (frame: 2124x2114-8-11) margins: 11, 45, 11, 11 minimum size: 608x585 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=1238,1226 maxtrack=0,0)


## Load image and check channels

In [6]:
im = imread('F:20220517_PS19_161_RL_Atto425_520xl/output_Left/round_RL018_b1_stitched_image.tif')

In [7]:
im.shape

(63, 5646, 7492)

In [8]:
viewer.add_image(im, name='dapi', blending='additive', colormap='bop blue', scale=[1,1,1])

<Image layer 'dapi' at 0x1cd97c9ed30>

## Nuclear segmentation on dapi

In [9]:
# Define cell model
path_to_model = 'F:cellpose_training_DAPI/2D/models/CP_DAPI_ISS' # edit path to ISS folder

model = models.CellposeModel(pretrained_model=path_to_model, gpu=True)

2022-05-23 14:09:23,359 [INFO] >>>> loading model F:cellpose_training_DAPI/2D/models/CP_DAPI_ISS
2022-05-23 14:09:23,659 [INFO] ** TORCH CUDA version installed and working. **
2022-05-23 14:09:23,659 [INFO] >>>> using GPU
2022-05-23 14:09:25,049 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2022-05-23 14:09:25,049 [INFO] >>>> model diam_labels =  32.439 (mean diameter of training ROIs)


In [10]:
# Cellpose parameters
cellpose_ini = {
   'channels': [0, 0],
   'batch_size': 16,
   'anisotropy': 1.0,
   'cellprob_threshold': 0,
   'flow_threshold': 0.5, #Useless in 3D
    'stitch_threshold': 0.5,
    'do_3D' : False,
    'progress' : True
}

In [11]:
masks, flows, styles = model.eval(im, **cellpose_ini)

2022-05-23 14:10:12,482 [INFO] multi-stack tiff read in as having 63 planes 1 channels
2022-05-23 14:20:10,144 [INFO] 100%|##########| 63/63 [09:43<00:00,  9.27s/it]
2022-05-23 14:20:10,206 [INFO] network run in 583.76s
2022-05-23 14:20:10,246 [INFO] No cell pixels found.
2022-05-23 14:20:10,288 [INFO] No cell pixels found.
2022-05-23 14:45:48,059 [INFO] stitching 63 planes using stitch_threshold=0.500 to make 3D masks
2022-05-23 14:48:36,042 [INFO] masks created in 1705.84s


In [12]:
# Check segmentation in napari
viewer.add_labels(masks)

<Labels layer 'masks' at 0x1cd9a992b20>

2022-05-23 14:55:42,924 [WARNING] Error drawing visual <Volume at 0x1cd067a0fd0>


2022-05-23 14:55:42,992 [WARNING] Error drawing visual <Volume at 0x1cd067a0fd0>


2022-05-23 14:55:43,514 [WARNING] Error drawing visual <Volume at 0x1cd067a0fd0>


2022-05-23 15:12:59,618 [WARNING] Error drawing visual <Volume at 0x1cd067a0fd0>


2022-05-23 15:13:00,194 [WARNING] Error drawing visual <Volume at 0x1cd067a0fd0>


In [ ]:
# Save segmentation as picture
output_file='F:20220517_PS19_161_RL_Atto425_520xl/output/dapi_segmentation'
io.save_masks(im, masks, flows, file_names=output_file, png=False, 
              tif=True, save_outlines=True, save_flows=True)

In [13]:
# Save mask array
np.save(file='F:20220517_PS19_161_RL_Atto425_520xl/output_Left/dapi_segmentation/masks,npy', 
       arr=masks)

In [ ]:
# Reload segmentation
dat = np.load('F:20220517_PS19_161_RL_Atto425_520xl/output/dapi_segmentation/masks.npy', 
              allow_pickle=True)

In [ ]:
viewer.add_labels(dat)